In [8]:
import os
import pandas as pd
from pandas import read_csv
from scipy.optimize import curve_fit
from matplotlib import pyplot as plt
import numpy as np
from scipy.optimize import minimize
import torch
from torch import autograd
from scipy.special import eval_legendre


In [2]:
# define configurational entropy
def s_config(x, s0, *si):
    """
    Compute the configurational entropy:
    
        s_config(x) = -k_B [ x log(x) + (1-x) log(1-x) ] *
                      [ (1 + w0) + sum_{i=1}^n (w_i * P_i(1 - 2x)) ].
    
    Here, the coefficients w_i are generated by the function `get_wi(i)`.

    Parameters
    ----------
    x : float
        Composition variable, must be in (0, 1) for the log terms to be valid.
    w0 : float
        The constant coefficient added to 1 in the second bracket.
    n : int
        The number of Legendre polynomial terms (i.e. the maximum i for which you compute w_i and P_i).

    Returns
    -------
    float
        Value of the configurational entropy at x.
    """
    k_B = 1.380649e-23
    # Ensure x is in the proper domain (0,1) to avoid math errors.
    if x <= 0.0 or x >= 1.0:
        # Return 0 or raise an error if x is out of (0,1)
        return 0.0

    # First bracket: -kB [ x ln(x) + (1-x) ln(1-x) ]
    bracket1 = -k_B * (x * np.log(x) + (1 - x) * np.log(1 - x))

    # Second bracket: (1 + w0) + sum_{i=1 to n} [w_i * P_i(1 - 2x)]
    bracket2 = (1.0 + s0)
    for i, si in enumerate(si, start=1):
        bracket2 += si * eval_legendre(i, 1.0 - 2.0 * x)

    return bracket1 * bracket2

In [3]:
def h_excess(x, *hi):
    # Ensure x is in the proper domain (0,1) to avoid math errors.
    if x <= 0.0 or x >= 1.0:
        # Return 0 or raise an error if x is out of (0,1)
        return 0.0

    # First bracket: -kB [ x ln(x) + (1-x) ln(1-x) ]
    bracket1 = x * (1 - x)

    # Second bracket: (1 + w0) + sum_{i=1 to n} [w_i * P_i(1 - 2x)]
    bracket2 = 0
    for i, hi in enumerate(hi, start=0):
        bracket2 += hi * eval_legendre(i, 1.0 - 2.0 * x)

    return bracket1 * bracket2

In [4]:
def s_vib(x, T, *ht_list):
    """
    Compute the vibrational entropy s_vib,M(x, T) based on the expansion of Theta_{E,M}(x)
    in Legendre polynomials:
    
        s_vib,M(x, T) = -3 x k_B * [
            log(1 - exp( -ThetaE(x)/T )) 
            - (ThetaE(x)/T) / (exp(ThetaE(x)/T) - 1 )
        ],
        
    where ThetaE(x) = sum_{i=0}^n [ ht_i * P_i(1 - 2x) ].

    Parameters
    ----------
    x : float
        Composition variable (commonly in the range 0 < x < 1).
    T : float
        Temperature (must be > 0).
    ht_list : list or array-like
        The coefficients [ht_0, ht_1, ..., ht_n] for the Legendre polynomial expansion
        of Theta_{E,M}(x).
    k_B : float, optional
        Boltzmann constant or chosen scaling factor. Defaults to 1.0 (dimensionless).

    Returns
    -------
    float
        The vibrational entropy s_vib,M for the given x and T.
    """
    k_B = 1.380649e-23
    # Basic checks (optional)
    if not (0 < x < 1):
        return 0.0
    if T <= 0:
        raise ValueError("Temperature T must be positive.")

    # 1) Compute Theta_{E,M}(x) by summing ht_i * P_i(1 - 2x).
    ThetaE = 0.0
    for i, ht_i in enumerate(ht_list):
        ThetaE += ht_i * eval_legendre(i, 1.0 - 2.0*x)

    # 2) Compute the bracket:
    #    bracket = log(1 - exp(-ThetaE/T)) - (ThetaE/T) * 1 / (exp(ThetaE/T) - 1)
    #    (Make sure to handle potential numerical issues if ThetaE/T is large, etc.)
    exp_factor = np.exp(-ThetaE / T)
    if exp_factor >= 1.0:
        # If ThetaE is negative or zero, you could return 0 or handle carefully
        return 0.0

    bracket = (
        np.log(1.0 - exp_factor)
        - (ThetaE / T) / (np.exp(ThetaE / T) - 1.0)
    )

    # 3) Combine with the prefactor -3 x k_B
    s_vib_val = -3.0 * x * k_B * bracket
    return s_vib_val

In [5]:
def h_vib(x, T, *ht_list):
    """
    Compute the vibrational enthalpy (or vibrational energy) h_vib,M(x, T) based on 
    an Einstein-like model:

        h_vib,M(x, T) = 3 x k_B * [
            (1/2) * ThetaE(x) + ThetaE(x) / (exp(ThetaE(x)/T) - 1)
        ],

    where ThetaE(x) = sum_{i=0}^n [ ht_i * P_i(1 - 2x) ].

    Parameters
    ----------
    x : float
        Composition variable (commonly in the range 0 < x < 1).
    T : float
        Temperature (must be > 0).
    ht_list : list or array-like
        The coefficients [ht_0, ht_1, ..., ht_n] for the Legendre polynomial expansion
        of Theta_{E,M}(x).
    k_B : float, optional
        Boltzmann constant or chosen scaling factor. Defaults to 1.0 (dimensionless).

    Returns
    -------
    float
        The vibrational enthalpy h_vib,M for the given x and T.
    """
    k_B = 1.380649e-23
    # Basic checks (optional)
    if not (0 < x < 1):
        return 0.0
    if T <= 0:
        raise ValueError("Temperature T must be positive.")

    # 1) Compute Theta_{E,M}(x) by summing ht_i * P_i(1 - 2x).
    ThetaE = 0.0
    for i, ht_i in enumerate(ht_list):
        ThetaE += ht_i * eval_legendre(i, 1.0 - 2.0*x)

    # 2) Compute the bracket:
    #    bracket = (1/2)*ThetaE + ThetaE / (exp(ThetaE/T) - 1)
    denom = np.exp(ThetaE / T) - 1.0
    if denom == 0.0:
        # This could happen if ThetaE/T is ~ 0 => handle carefully
        # A small limit or approximate expansion might be used here.
        return 0.0  # or some limit for small ThetaE/T

    bracket = 0.5 * ThetaE + (ThetaE / denom)

    # 3) Combine with the prefactor 3 x k_B
    h_vib_val = 3.0 * x * k_B * bracket
    return h_vib_val

In [10]:
loc = r"C:\UM\Study_Material\Capstone\Graphite_OCP_Archie"
ref_temp = 298
path = os.path.join(loc, f"Graphite_lithiation_{ref_temp}.csv")
data_ref = pd.read_csv(path, names=['Li_X', 'Potential'])
x_ref = data_ref['Li_X'].values
y_ref = data_ref['Potential'].values